In [2]:
pip install geopy

     |████████████████████████████████| 112kB 22.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geocoder

     |████████████████████████████████| 102kB 5.6MB/s ta 0:00:011
     |████████████████████████████████| 829kB 5.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


Scrapping Data

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [11]:
# append the data into the list
for row in soup.find_all("div", class_="div-col columns column-width")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [45]:
# create a new DataFrame from the list
la_df = pd.DataFrame({"Neighborhood": neighborhoodList})

la_df.head()

,Neighborhood
0,Angelino Heights[1]
1,Angeles Mesa
2,Angelus Vista
3,Arleta[2][1]
4,Arlington Heights[2]


In [46]:
for i in list(range(1,58)):
    a = '[' + str(i) + ']'
    la_df['Neighborhood'] = la_df['Neighborhood'].map(lambda x: x.rstrip(a))


In [47]:
la_df.head()

,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vista
3,Arleta
4,Arlington Heights


In [49]:
la_df.shape

(102, 1)

Get the geographical coordinates

In [50]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [51]:
coords = [ get_latlng(neighborhood) for neighborhood in la_df["Neighborhood"].tolist() ]

In [52]:
coords

[[34.07029000000006, -118.25479999999999],
 [32.764074303732954, -116.98617130920526],
 [34.08757488206556, -118.26715643600681],
 [34.24905000000007, -118.43348999999995],
 [34.03989000000007, -118.32541999999995],
 [33.995992784459574, -118.18689217643083],
 [34.11970000000008, -118.25886999999994],
 [34.021570000000054, -118.36764999999997],
 [34.01043000000004, -118.33665999999994],
 [34.07044548379267, -118.20070982233794],
 [34.07044548379267, -118.20070982233794],
 [34.10915025473649, -118.32045034087673],
 [34.08361000000008, -118.43482999999998],
 [34.093863023638136, -118.42912206311348],
 [34.09653268819107, -118.40329514585207],
 [34.11793000000006, -118.44608999999997],
 [34.09544836979449, -118.42608686997163],
 [34.06147000000004, -118.39871999999997],
 [34.063720000000046, -118.26459999999997],
 [34.05138000000005, -118.40132999999997],
 [34.04004000000003, -118.21049999999997],
 [34.06626000000006, -118.47032999999999],
 [33.98805503453503, -118.27720988461618],
 [33.9

In [53]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [55]:
la_df['Latitude'] = df_coords['Latitude']
la_df['Longitude'] = df_coords['Longitude']

In [57]:
print(la_df.shape)
la_df.head(10)

(102, 3)


,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070290,-118.254800
1,Angeles Mesa,32.764074,-116.986171
2,Angelus Vista,34.087575,-118.267156
3,Arleta,34.249050,-118.433490
4,Arlington Heights,34.039890,-118.325420
5,Arts District,33.995993,-118.186892
6,Atwater Village,34.119700,-118.258870
7,Baldwin Hills,34.021570,-118.367650
8,Baldwin Hills/Crenshaw,34.010430,-118.336660
9,Baldwin Village,34.070445,-118.200710


In [58]:
la_df.to_csv("la_df.csv", index=False)

Create a map

In [60]:
address = 'Los Angeles, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of LA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of LA 34.0536909, -118.2427666.


In [61]:
map_la = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_la)  
    
map_la

In [62]:
map_la.save('map_la.html')

Use the Foursquare API

In [63]:
CLIENT_ID = 'ZYVHZAJWTHSN3ZOJXQSYC5EH1EFVDDD1AK0BCGJMDZVX4WOE' # your Foursquare ID
CLIENT_SECRET = 'AJO0APR3QG5GH40ZNDMLIUN0TMEGQMB2AMXF2GEORM3OTRL3' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZYVHZAJWTHSN3ZOJXQSYC5EH1EFVDDD1AK0BCGJMDZVX4WOE
CLIENT_SECRET:AJO0APR3QG5GH40ZNDMLIUN0TMEGQMB2AMXF2GEORM3OTRL3


In [64]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [65]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8648, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Angelino Heights,34.07029,-118.2548,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
1,Angelino Heights,34.07029,-118.2548,Guisados,34.070262,-118.250437,Taco Place
2,Angelino Heights,34.07029,-118.2548,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
3,Angelino Heights,34.07029,-118.2548,Button Mash,34.073975,-118.251507,Arcade
4,Angelino Heights,34.07029,-118.2548,Konbi,34.075383,-118.253893,Japanese Restaurant


In [66]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
East Los Angeles,100,100,100,100,100,100
Angeles Mesa,42,42,42,42,42,42
Angelino Heights,100,100,100,100,100,100
Angelus Vista,100,100,100,100,100,100
Arleta,56,56,56,56,56,56
Arlington Heights,100,100,100,100,100,100
Arts District,47,47,47,47,47,47
Atwater Village,100,100,100,100,100,100
Baldwin Hills,90,90,90,90,90,90


In [67]:
venues_df['VenueCategory'].unique()[:50]

array(['Performing Arts Venue', 'Taco Place', 'Coffee Shop', 'Arcade',
       'Japanese Restaurant', 'American Restaurant', 'Beer Store',
       'Massage Studio', 'Art Gallery', 'Sake Bar', 'Bar',
       'Latin American Restaurant', 'Park', 'Bookstore',
       'Vegetarian / Vegan Restaurant', 'Pool', 'Music Venue',
       'Deli / Bodega', 'Food Truck', 'Mexican Restaurant', 'Pet Store',
       'Pizza Place', 'Café', 'Wine Bar', 'Cocktail Bar',
       'Sandwich Place', 'Gym / Fitness Center', 'Circus', 'Bakery',
       'Health Food Store', 'Breakfast Spot', 'Gastropub',
       'Baseball Stadium', 'Theater', 'Grocery Store', 'Art Museum',
       'Scenic Lookout', 'Plaza', 'Concert Hall', 'Farmers Market',
       'Clothing Store', 'Sports Bar', 'Museum', 'Fried Chicken Joint',
       'Yoga Studio', 'Thai Restaurant', 'Ice Cream Shop',
       'Convenience Store', 'Cajun / Creole Restaurant',
       'Filipino Restaurant'], dtype=object)

Analyze Each Neighborhood

In [68]:
la_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

print(la_onehot.shape)
la_onehot.head()

(8648, 376)


,Neighborhoods,ATM,Acai House,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service

In [69]:
la_grouped = la_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(la_grouped.shape)
la_grouped

(102, 376)


,Neighborhoods,ATM,Acai House,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service

In [70]:
len(la_grouped[la_grouped["Hotel"] > 0])

45

In [71]:
la_hotel = la_grouped[["Neighborhoods","Hotel"]]

In [73]:
la_hotel.head()

,Neighborhoods,Hotel
0,East Los Angeles,0.000000
1,Angeles Mesa,0.047619
2,Angelino Heights,0.000000
3,Angelus Vista,0.000000
4,Arleta,0.000000


Cluster Neighborhoods

In [74]:
kclusters = 3

la_clustering = la_hotel.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_clustering)

kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [75]:
la_merged = la_hotel.copy()

la_merged["Cluster Labels"] = kmeans.labels_

In [76]:
la_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
la_merged.head()

,Neighborhood,Hotel,Cluster Labels
0,East Los Angeles,0.000000,1
1,Angeles Mesa,0.047619,0
2,Angelino Heights,0.000000,1
3,Angelus Vista,0.000000,1
4,Arleta,0.000000,1


In [77]:
la_merged = la_merged.join(la_df.set_index("Neighborhood"), on="Neighborhood")

print(la_merged.shape)
la_merged.head() 

(102, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,East Los Angeles,0.000000,1,34.033470,-118.159090
1,Angeles Mesa,0.047619,0,32.764074,-116.986171
2,Angelino Heights,0.000000,1,34.070290,-118.254800
3,Angelus Vista,0.000000,1,34.087575,-118.267156
4,Arleta,0.000000,1,34.249050,-118.433490


In [78]:
print(la_merged.shape)
la_merged.sort_values(["Cluster Labels"], inplace=True)
la_merged

(102, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
33,Central City,0.040000,0,37.377620,-121.985227
22,Brentwood,0.020833,0,34.066260,-118.470330
80,Hollywood,0.050000,0,34.101560,-118.337080
82,Hollywood Hills,0.040000,0,34.105240,-118.320650
18,Beverly Hills Post Office,0.040000,0,34.061470,-118.398720
83,Hollywood Hills West,0.041096,0,34.120060,-118.350900
73,Harbor City,0.030000,0,33.797810,-118.302480
64,Franklin Hills,0.030000,0,34.103432,-118.349341
87,Jewelry District,0.040000,0,34.047920,-118.258010
63,Flower District,0.040000,0,34.047920,-118.258010


In [80]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
map_clusters.save('map_clusters.html')

Result

In [82]:
la_merged.loc[la_merged['Cluster Labels'] == 0]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
33,Central City,0.040000,0,37.377620,-121.985227
22,Brentwood,0.020833,0,34.066260,-118.470330
80,Hollywood,0.050000,0,34.101560,-118.337080
82,Hollywood Hills,0.040000,0,34.105240,-118.320650
18,Beverly Hills Post Office,0.040000,0,34.061470,-118.398720
83,Hollywood Hills West,0.041096,0,34.120060,-118.350900
73,Harbor City,0.030000,0,33.797810,-118.302480
64,Franklin Hills,0.030000,0,34.103432,-118.349341
87,Jewelry District,0.040000,0,34.047920,-118.258010
63,Flower District,0.040000,0,34.047920,-118.258010


In [85]:
la_merged.loc[la_merged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
71,Griffith Park,0.000000,1,34.123270,-118.299900
69,Granada Hills,0.000000,1,34.263940,-118.497940
0,East Los Angeles,0.000000,1,34.033470,-118.159090
67,Glassell Park,0.000000,1,34.119220,-118.230440
62,Florence,0.000000,1,33.974750,-118.245390
61,Financial District,0.010000,1,37.795780,-122.400480
60,"Filipinotown, Historic",0.000000,1,34.053610,-118.245500
58,Fairfax,0.000000,1,34.076100,-118.361220
57,Faircrest Heights,0.000000,1,34.090770,-118.078365
68,Gramercy Park,0.000000,1,34.033900,-118.312580


In [84]:
la_merged.loc[la_merged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
15,Beverly Crest,0.090909,2,34.096533,-118.403295
14,Benedict Canyon,0.096774,2,34.093863,-118.429122
84,Holmby Hills,0.070000,2,34.052901,-118.423903
65,Gallery Row,0.071429,2,34.100800,-118.426140
90,Ladera,0.157895,2,34.081567,-118.423414
17,Beverly Grove,0.086957,2,34.095448,-118.426087
20,Beverlywood,0.070000,2,34.051380,-118.401330
47,East Gate Bel Air[citation needed,0.076923,2,34.083610,-118.434830
13,"Bel Air, Bel-Air or Bel Air Estates",0.076923,2,34.083610,-118.434830


In [87]:
la_merged.groupby(['Cluster Labels'])['Hotel',].mean()

,Hotel
Cluster Labels,
0,0.034436
1,0.002121
2,0.088645


Most of hotels are concentrated in the western part of LA, with the hightest number in cluster 2. Also, downtown and sightseeing areas hold some hotels when we see cluster 0. On the other hand, cluster 1 has very low number of hotels in the area. We can guess that this area is residential area. \
Hotels in cluster 2 are likely to suffer from intense competition due to over supply and high concentration of hotels. Also, Hotels in cluster 1 would face low demand of hotels. Therefore, this project recommends property developers to open new hotels in neighborhoods in cluster 1 with moderate competition.